# Introduction
This notebook serves as an example for the process of the analysis of semantic similarity. Semantic similarity requires not simply comparing text, but comparing what the text is supposed to mean. Even people can have disagreements over this, so computing something that abstract is a complex task.

A concept of vital importance is text embedding. Text is relatively hard to compare; comparing numbers is significantly faster than comparing strings, the latter possibly requiring iterating over each character to check it. Transforming the text into a numerical representation brings a lot of efficiency, and is an important first step. There are multiple ways to go about this.

The simplest way is called \textit{bag of words}. The entire vocabulary of the library of text we want to compare is numerically allocated; each word is given an entry in a simple lookup table. A sentence can then be vectorized, each vector being the same length as the lookup table, and each index holding the count of how often the word assigned that number appears in the sentence. Comparing these vectors then essentially compares the overlap between words used in sentences, assuming that the same words lead to the same meanings.

Of course, as the 'bag' implies, we lose order, a very important part of semantic meaning. Additionally, words may be functionally similar, but be different counts; two sentences each using different synonyms would be very different in this theory. 'Greater' could be stemmed to 'great', but the same is harder for 'better' and 'good'. What about 'terrific' and 'great'? We could look up synonyms for each word, but this quickly becomes very inefficient. Then there's the question of how relevant a word is; a 'I have a good book' is very different from a 'I have a good car', after all, despite being 80% "similar".

This relevancy problem can be diminished using TFIDF, or term frequency–inverse document frequency, to introduce a weighting. The more often a term shows up in a document, which is any arbitrary amount of text, the more relevant it is. The more documents that term shows up in, however, the less relevant. The sentences above may be reduced to 'good book' and 'good car', for example, based on the exact implementation. 

A more complex solution is to use a continuous bag of words such as described in the Word2Vec model as originally patented by Google, an implementation of which exists in the open source Gensim package. The concept is similar to a normal bag of words, but instead of each word apart, we look at the surrounding words. Each word is paired with its surroundings, creating a structure that allows for the prediction of words: using the above sentences again, 'good' can predict 'a', 'book', or 'car'. Then, TFIDF could be applied to determine the odds of predicting each word, or these weights can be determined more accurately using a neural network, as described in the original article. The Gensim package uses the latter method. The (abstracted) weights can be vectorized, and these are then used much like the simpler embedding can be.

To compare sentences, the embedded word vector can be taken for each word in the sentence, taking the average. Two averaged vectors can then be compared to determine how similar they are. This solves the relevancy and synonym problems, but does not preserve order; some context from sentence structure is lost. An expansion on the Word2Vec method, the Doc2Vec method, designed by the same authors, proposes to solve this by taking the word vectors in a document, and adding a 'phantom word'; the resulting so called document vector is then unique for that document, in an attempt to remember or at least account for the entire document context. This also has an implementation built in Gensim.

How long the resulting vector is, depends on the implementation. The Gensim implementations use a vector size of 100 by default. Larger vectors can capture more details, but make computation more expensive. In our case, 100 was an apt choice 

In the next cell, we load versions of these three models trained on the data provided to us, to show their capability. Additionally, a pretrained Google model known as the Universal Sentence Encoder will be shown.

# Embedding Models

In [8]:
from gensim.models import word2vec, doc2vec
from embed import universal_sentence_encoder as use_model


# Load the pretrained models
w2v_model = word2vec.Word2Vec.load("models/w2vmodel.mod")
d2v_model = doc2vec.Doc2Vec.load("models/doc2vec.model")
# tfidf model?


# Classification

The above models and instruction clarify how to embed text, acquiring vectors for the relevant documents. It mentions comparing these vectors, but not exactly how. There are, again, multiple ways to do this. The simplest way is to calculate the cosine similarity between them, essentially imagining we have position vectors, and checking the angle between them. This is relatively easy to compute, intuitive, and gives a result between 0 and 1, making it very easy to use; set or train a proper threshold, and if the cosine similarity is above that threshold, the sentences carry the same semantic intent.

A complexer but significantly more accurate way of comparing is through the use of a neural network. Starting with more-or-less random predictions, the correct answers are used to calculate how wrong the network's prediction was, which is then used to update it, improving the next prediction. Through repetition, the network thus learns how to tell which vectors are supposed to be 'similar', learning to classify them. We implemented a relatively simple neural network, that takes as input two concatenated vectors; the two sentence vectors we want to compare. One hidden layer, of equal size to the input layer, with as activation function the rectified linear activation function, and finally a sigmoid function for the output layer of size 1. This output is simply a 0 or 1, signifying whether the sentences are the same or not.

The training of this second method is rather computationally expensive, though training doesn't necessarily have happen more than once. However, it also requires a large amount of annotated data; a large list of sentence combinations of which it is known if they are semantically the same or not. Additionally, this data should be very varied, as scenarios not adequately present in the training data will be hard to predict. This data being accurate is also of vital importance for accuracy, meaning it will have to be done, or at least verified, by hand. Our data consisted of about 400000 pairs of questions, meaning the network is likely to be better at classifying questions than other sentence structures. While this machine learning is not applicable in every situation, the prerequisites are something to keep in mind.

Below we load our pretrained model, after which the requirements for analysis are present, and some examples can be run.

# Neural Network

In [12]:
from tensorflow import keras
from classifiers import binaryclassification as bc
from embed import doc2vec

# model = keras.models.load_model('models/...')

sentence1 = "Is X the same as Y?"
sentence2 = "When is X the same as Y?"

w2v_vector = bc.vectorize_w2v(w2v_model.wv, sentence1, sentence2)
d2v_vector = doc2vec.doc2vec(d2v_model, [sentence1], [sentence2])
use_vector = use_model.encode(sentence1) + use_model.encode(sentence2)


tf.Tensor(
[ 0.08101482 -0.2019411  -0.01571975  0.14588054  0.0861248  -0.00461761
  0.13665743 -0.08589698  0.06596185  0.15349     0.06258003  0.0618754
  0.00780827  0.02202326  0.00156243 -0.13307479 -0.05229007  0.11145592
  0.03328857 -0.00193182  0.05196795 -0.04891817  0.10904198  0.12596294
  0.06784732  0.05713417  0.0007825   0.06168942 -0.00151605  0.00711261
 -0.01023459  0.11996448  0.07660645 -0.07809665  0.07484739 -0.07300623
  0.07918902 -0.10433002  0.08221419  0.069256    0.08242436 -0.06219982
  0.02070184  0.07483666  0.19298664 -0.07806981 -0.02932242 -0.09839576
  0.13108103  0.04633947 -0.06870607  0.21059522  0.10954858 -0.02387153
 -0.00523979  0.05046488 -0.10549062 -0.08490714 -0.10892796 -0.01983385
 -0.10960896 -0.00747574 -0.03746359 -0.08899061  0.15564816 -0.20212987
 -0.03029954 -0.06273824  0.00751871 -0.04651058  0.03027163 -0.06498927
  0.04003879 -0.00468695 -0.02704884  0.01310218 -0.0405383  -0.03496305
 -0.11583926  0.14845836  0.03391928 -0.0

In [ ]:
print("W2V says: ", model_w2v.predict(w2v_vector))
print("W2V says: ", model_d2v.predict(d2v_vector))
print("W2V says: ", model_tfidf.predict(tfidf_vector))
print("W2V says: ", model_use.predict(use_vector))